***
<p style="text-align: center;"><span style='font-family: "Courier New", courier; color: rgb(184, 49, 47);'><strong><span style="font-size: 36px;">Generation of Road Data Set for Segmentation</span></strong></span></p>

***


# Libraries

In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from urllib.request import urlopen
import folium
import base64
from IPython.display import HTML, display, clear_output
# Rasterio is not a default library, may require installation
try:
    import rasterio
except ModuleNotFoundError:
    !pip install rasterio
    import rasterio

# User Parameters

- `API_key` Google Map Static API Key is unique to each user
- `project_name` Name of the project folder that will be created if not exists
- `minLat, maxLat, minLon, maxLon` Geographic bounderies of the area of interest
- `image_size` Size of the image to be downloaded
- `metadata` Google Map Static API Key is unique to each user
- `rectify` To rectify satellite and mask images and produce GeoTiff files (True/False) 

In [ ]:
# Create project folders

# Project working directory and sub-directories
project_name = r"googlemap"

# Zoom levels and number of images to be downloaded
metadata = [(14, 10, "#1A5276"), # Zoom level 14: 10 images
            (15, 10, "#78281F"), # Zoom level 15: 10 images
            (16, 10, "#0E6655"), # Zoom level 16: 10 images
            (17, 10, "#F1C40F")  # Zoom level 17: 10 images
           ]
zoom_levels = [level[0] for level in metadata]

pwd = Path(project_name)
satellite_dir = pwd.joinpath("sat")
mask_dir      = pwd.joinpath("mask")

# Google Map Static API KEY (specific to each user)
API_key = "INSERT YOUR API KEY HERE"

# Boundaries of the area of interest (in degrees)
minLat, maxLat, minLon, maxLon = 41.0, 41.2, 28.75, 29.5

# Pixel size of satellite and mask images
# Note: Google Map does not allow an image size greater than 640X640
# though it does not give an error about it.
image_size = 512 # 512x512 image size

# Rectifying satellite images
# True  : produces GeoTiff file of Google Map images
# False : will not produces GeoTiff files, though log file contains coordinate information anyway
rectify = True

# ------------------------------------------------------------------------------
# Create project working directory

if pwd.is_dir()==True:
    print("Project working directory exists...")
    if satellite_dir.is_dir() == False:
        satellite_dir.mkdir()
    if mask_dir.is_dir() == False:
        mask_dir.mkdir()
    for zoom in zoom_levels:
        satellite_zoom_dir = satellite_dir.joinpath(str(zoom))
        mask_zoom_dir      = mask_dir.joinpath(str(zoom))
        if satellite_zoom_dir.is_dir() == False:
            satellite_zoom_dir.mkdir()
        if mask_zoom_dir.is_dir() == False:
            mask_zoom_dir.mkdir()  
else:
    pwd.mkdir()
    print("Project working directory is created | PWD:", pwd.absolute())
    satellite_dir.mkdir()
    mask_dir.mkdir()
    for zoom in zoom_levels:
        satellite_zoom_dir = satellite_dir.joinpath(str(zoom))
        mask_zoom_dir      = mask_dir.joinpath(str(zoom))
        satellite_zoom_dir.mkdir()
        mask_zoom_dir.mkdir()
# ------------------------------------------------------------------------------
# Log file of project
log = open(project_name+"/log.txt","wt")
print("Project Name:", project_name, file=log)
print("Zoom levels selected: ", zoom_levels, file=log)
print("Area of interest:", (minLat, maxLat, minLon, maxLon), file=log)
print("Image size:", image_size, file=log)

# Google Map API JSON Styling
url =  ["https://maps.googleapis.com/maps/api/staticmap?center={},{}","zoom={}","size=640x640","scale=1",
        "maptype={}",
        "style=element:geometry%7Ccolor:0x000000",
        "style=element:labels.icon%7Cvisibility:off",
        "style=element:labels.text.fill%7Ccolor:0x000000",
        "style=element:labels.text.stroke%7Ccolor:0x000000",
        "style=feature:administrative%7Cvisibility:off",
        "style=feature:administrative%7Celement:geometry%7Ccolor:0x000000",
        "style=feature:administrative.country%7Celement:labels.text.fill%7Ccolor:0x000000",
        "style=feature:administrative.land_parcel%7Cvisibility:off",
        "style=feature:administrative.locality%7Celement:labels.text.fill%7Ccolor:0x000000",
        "style=feature:landscape%7Celement:geometry.fill%7Ccolor:0x000000",
        "style=feature:poi%7Celement:geometry.fill%7Ccolor:0x000000",
        "style=feature:poi%7Celement:labels.text.fill%7Ccolor:0x000000",
        "style=feature:poi.park%7Cvisibility:off",
        "style=feature:poi.park%7Celement:geometry%7Ccolor:0x000000",
        "style=feature:poi.park%7Celement:labels.text.fill%7Ccolor:0x000000",
        "style=feature:poi.park%7Celement:labels.text.stroke%7Ccolor:0x000000",
        "style=feature:road%7Celement:geometry.fill%7Ccolor:0xffffff",
        "style=feature:road%7Celement:labels%7Ccolor:0x000000%7Cvisibility:off",
        "style=feature:road%7Celement:labels.text.fill%7Ccolor:0xffffff",
        "style=feature:road.arterial%7Celement:geometry%7Ccolor:0xffffff",
        "style=feature:road.highway%7Celement:geometry%7Ccolor:0xffffff",
        "style=feature:road.highway.controlled_access%7Celement:geometry%7Ccolor:0xffffff",
        "style=feature:road.local%7Celement:labels.text.fill%7Ccolor:0xffffff",
        "style=feature:road.local.trail%7Celement:geometry%7Ccolor:0x000000"
        "style=feature:transit%7Cvisibility:off",
        "style=feature:transit%7Celement:labels.text.fill%7Ccolor:0x000000",
        "style=feature:water%7Cvisibility:off",
        "style=feature:water%7Celement:geometry%7Ccolor:0x000000",
        "style=feature:water%7Celement:labels.text.fill%7Ccolor:0x000000",
        "key={}",
        ]

# Function for calculating corner coordinates
def get_corners(latitude, longitude, image_size, zoom):
    """
    Function that calculates the geographic coordinates of corner pixels in WGS84

    input:
        latitude   : Latitude of central pixel in degrees
        longitude  : Longitude of central pixel in degrees
        image_size : image size (ex: 512 for 512x512 image)
        zoom       : zoom level in Google Map API
    
    return:
        latitude and longitude of NE, NW, SW and SE corners respectively
    """
    corner_pixel_coordinates = np.array([[image_size, 0],          # North-East
                                         [0, 0],                   # North-West
                                         [0, image_size],          # South-West
                                         [image_size, image_size], # South-East
                                        ])
    latitude_scale = np.cos(np.deg2rad(latitude))
    dx_degrees = 360 / 2**(zoom + 8)
    dy_degrees = 360 / 2**(zoom + 8) * latitude_scale
    lon = longitude + dx_degrees * (corner_pixel_coordinates[:,0] - image_size / 2)
    lat = latitude  - dy_degrees * (corner_pixel_coordinates[:,1] - image_size / 2)
    return (lat, lon)

Project working directory is created | PWD: /content/googlemap


# Downloading Images and Visualization

In [ ]:
folium_map = folium.Map(location=[(minLat+maxLat)/2,(minLon+maxLon)/2],
                        zoom_start=10,
                        tiles="cartodbpositron") #  CartoDB dark_matter

starting_pixel = (640-image_size)//2
ending_pixel   = starting_pixel + image_size

print(file=log)
print("-"*20*13,file=log)
print(("{:>20}"*13).format("Image ID", "Satellite Image", "Mask Image", 
                         "Center Latitude", "Center Longitude",
                         "NE Latitude", "NE Longitude",
                         "NW Latitude", "NW Longitude",
                         "SW Latitude", "SW Longitude",
                         "SE Latitude", "SE Longitude"), file=log)
print("-"*20*13,file=log)
for index in metadata:
    zoom       = index[0]
    num_images = index[1]
    zoom_color = index[2]
    satellite_zoom_dir = satellite_dir.joinpath(str(zoom))
    mask_zoom_dir      = mask_dir.joinpath(str(zoom))
    for i in tqdm(range(0, num_images), desc='| Zoom level:' + str(zoom), position=0, leave=True):
        # Generate random coordinates inside the boundaries
        randomLat  = np.random.uniform(low=minLat, high=maxLat)
        randomLon  = np.random.uniform(low=minLon, high=maxLon)
        
        # Satellite view
        satellite_filename = "sat_{}_zoom{}.png".format(i,zoom)
        satellite_view = Image.new('RGB', (image_size, image_size))
        url_formatted = "&".join(url).format(randomLat, randomLon, zoom, "satellite", API_key)
        imx = Image.open(urlopen(url_formatted)).convert("RGB")
        satellite_view = imx.crop((starting_pixel, starting_pixel, ending_pixel, ending_pixel))
        satellite_view.save(satellite_zoom_dir.joinpath(satellite_filename))
        
        # Roadmap view
        roadmap_filename = "mask_{}_zoom{}.png".format(i,zoom)
        roadmap_view = Image.new('RGB', (image_size, image_size))
        url_formatted = "&".join(url).format(randomLat, randomLon, zoom, "roadmap", API_key)
        imx = Image.open(urlopen(url_formatted)).convert("RGB")
        roadmap_view = imx.crop((starting_pixel, starting_pixel, ending_pixel, ending_pixel))
        roadmap_view.save(mask_zoom_dir.joinpath(roadmap_filename))

        encoded = base64.b64encode(open(satellite_zoom_dir.joinpath("sat_0_zoom"+str(zoom)+".png"), 'rb').read())
        html = '<img src="data:image/png;base64,{}">'.format
        iframe = folium.IFrame(html(encoded.decode('UTF-8')), width=256, height=256)
        marker = folium.CircleMarker(location   = [randomLat, randomLon],
                                     radius     = 1,
                                     popup= folium.Popup(iframe, max_width=256), # "Image_" + str(i),
                                     color      = zoom_color,
                                     fill       = True,
                                     fill_color = zoom_color)
        marker.add_to(folium_map)
        
        # Calculate coordinates of corner pixels
        corners    = get_corners(randomLat, randomLon, image_size, zoom)
        coordinates = [
                      [corners[0][0], corners[1][0]],
                      [corners[0][1], corners[1][1]],
                      [corners[0][2], corners[1][2]],
                      [corners[0][3], corners[1][3]],
                      [corners[0][0], corners[1][0]]
                      ]
        borders=folium.PolyLine(locations = coordinates,
                                weight    = 2,
                                color     = zoom_color)
        folium_map.add_child(borders)
        
        print(("{:>20}"*3 + "{:20.8f}"*10).format(str(i), satellite_filename, roadmap_filename,
                                                  randomLat, randomLon,
                                                  corners[0][0], corners[1][0],
                                                  corners[0][1], corners[1][1],
                                                  corners[0][2], corners[1][2],
                                                  corners[0][3], corners[1][3]), file=log)
        if rectify==True:
            minlon, minlat, maxlon, maxlat = np.sort(corners[1])[0], np.sort(corners[0])[0], np.sort(corners[1])[-1], np.sort(corners[0])[-1]
            # Rectify Images Using Rasterio
            satellite_data_temp = rasterio.open(satellite_zoom_dir.joinpath(satellite_filename), 'r')
            satellite_data      = satellite_data_temp.read()
            transform = rasterio.transform.from_bounds(minlon, minlat, maxlon, maxlat, satellite_data.shape[1], satellite_data.shape[2])
    
            with rasterio.open(satellite_zoom_dir.joinpath(satellite_filename.split(".")[0] + ".tiff"), 'w', driver='GTiff',
                               width=satellite_data.shape[1], height=satellite_data.shape[2],
                               count=3, dtype=satellite_data.dtype, nodata=0,
                               transform=transform) as dst:
                dst.write(satellite_data)
            
            mask_data_temp = rasterio.open(mask_zoom_dir.joinpath(roadmap_filename), 'r')
            mask_data      = mask_data_temp.read()
            transform = rasterio.transform.from_bounds(minlon, minlat, maxlon, maxlat, mask_data.shape[1], mask_data.shape[2])
            with rasterio.open(mask_zoom_dir.joinpath(roadmap_filename.split(".")[0] + ".tiff"), 'w', driver='GTiff',
                               width=mask_data.shape[1], height=mask_data.shape[2],
                               count=3, dtype=mask_data.dtype, nodata=0,
                               transform=transform) as dst:
                dst.write(mask_data)
        # Display ground coverage
        if (i+1)%10 == 0:
            clear_output(wait=True)
            display(HTML(folium_map._repr_html_()))
clear_output(wait=True)
display(HTML(folium_map._repr_html_()))
folium_map.save(project_name+ "/ground_coverage.html")
print("-"*20*13,file=log)
log.close()